# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Top packages - stdlib, bottom packages - 3rd party
import re
import os
import glob
import time
import json
import csv
import traceback
import numpy as np
import pandas as pd
import cassandra

#### Creating list of filepaths to process original event csv data files

In [2]:
start_time = time.time()

# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
     # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# Code below is optimized in another file with dataframes
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### Getting spread statistics to optimize choice of clustering columns

Spread statistics (value_counts, mode, variance, standard deviation) provide an idea of how the data is distributed.
For text columns, we can compute mode and value_counts

In [5]:
df = pd.read_csv('event_datafile_new.csv')
headers = ['itemInSession', 'userId', 'sessionId', 'artist']

stats_dict = { header : [len(df[header].value_counts()), df[header].mode()[0]] for header in headers}
pd.DataFrame(stats_dict, index=['No of distinct values (value counts)', 'Most frequent occurrence (mode)'])


,itemInSession,userId,sessionId,artist
No of distinct values (value counts),123,96,776,3148
Most frequent occurrence (mode),1,49,589,Coldplay


## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
                    CREATE KEYSPACE IF NOT EXISTS udacity 
                    WITH REPLICATION = 
                    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Create Table Queries

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


For query 1, we create a table with just the 5 columns needed from the csv file. 
To uniquely identify a row, we need session_id and item_in_session columns as PRIMARY KEY. Here session_id is the PARTITION KEY and item_in_session is the CLUSTERING KEY.

In [9]:
create_query1 = """
                CREATE TABLE IF NOT EXISTS songplay_session (
                    session_id int,
                    item_in_session int,
                    artist text,
                    song text,
                    song_length float,
                    PRIMARY KEY (session_id, item_in_session)
                );
                """

#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

This query requires a table with 7 columns fetched from the csv file.  
To uniquely identify a row, we use user_id, session_id and item_in_session columns as PRIMARY KEY. Here user_id and session_id combined form the PARTITION KEY while item_in_session is the CLUSTERING KEY.

In [10]:
create_query2 = """
                CREATE TABLE IF NOT EXISTS songplay_user_session (
                    user_id int,
                    session_id int,
                    item_in_session int,
                    artist text,
                    song text,
                    first_name text,
                    last_name text,
                    PRIMARY KEY ((user_id, session_id), item_in_session)
                ) 
                WITH CLUSTERING ORDER BY (item_in_session asc);
                """


#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


The query requires 2 columns to select (first name and last name), and in addition, a PRIMARY KEY.

To uniquely identify a row, we use the song and user_id columns as the PRIMARY KEY. The song is the PARTITION KEY and user_id is the CLUSTERING KEY.

In [11]:
create_query3 = """
                CREATE TABLE IF NOT EXISTS user_song_history (
                    song text,
                    user_id int,
                    first_name text,
                    last_name text,
                    PRIMARY KEY (song, user_id)
                );
                """

Creating the Tables

In [12]:
try:
    session.execute(create_query1);
    session.execute(create_query2);
    session.execute(create_query3);
except Exception:
    traceback.print_exc()

#### Insert into table 1 songplay_session

In [13]:
# Read csv file and insert from data into table 1 for query 1
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songplay_session (session_id, item_in_session, artist, song, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

Query 1: artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [14]:
## SELECT statement to verify the data was entered into the table
select_query1 = """ SELECT artist, song, song_length
                    FROM songplay_session 
                    WHERE session_id = 338
                    AND item_in_session = 4;
                """
try:
    rows = session.execute(select_query1)
except Exception:
    traceback.print_exc()
    
for row in rows:
    print ((row.artist, row.song, "{:.2f} seconds".format(row.song_length)))

('Faithless', 'Music Matters (Mark Knight Dub)', '495.31 seconds')


#### Insert into table 2 songplay_user_session

In [15]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# Read csv file and insert from data into table 2 for query 2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songplay_user_session (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s);"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


Query 2: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [16]:
## SELECT statement to verify the data was entered into the table
select_query2 = """ SELECT artist, song, first_name, last_name
                    FROM songplay_user_session 
                    WHERE user_id = 10
                    AND session_id = 182
                    ORDER BY item_in_session;
                """
try:
    rows = session.execute(select_query2)
except Exception:
    traceback.print_exc()
    
for row in rows:
    print ((row.artist, row.song, row.first_name + " " + row.last_name))

('Down To The Bone', "Keep On Keepin' On", 'Sylvie Cruz')
('Three Drives', 'Greece 2000', 'Sylvie Cruz')
('Sebastien Tellier', 'Kilometer', 'Sylvie Cruz')
('Lonnie Gordon', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie Cruz')


#### Insert into Table 3 user_song_history

In [17]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Read csv file and insert from data into table 3 for query 3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_song_history (song, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))


Query 3: every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [18]:
## SELECT statement to verify the data was entered into the table
select_query3 = """ SELECT first_name, last_name
                    FROM user_song_history 
                    WHERE song = 'All Hands Against His Own';
                """
try:
    rows = session.execute(select_query3)
except Exception:
    traceback.print_exc()
    
for row in rows:
    print (row.first_name + " " + row.last_name)


Jacqueline Lynch
Tegan Levine
Sara Johnson


In [19]:
end_time = time.time()
print(f"Time taken to read, insert and select queries: {end_time - start_time} s")

Time taken to read, insert and select queries: 30.68772554397583 s


### Drop the tables before closing out the sessions

In [20]:
drop_query1 = """ DROP TABLE IF EXISTS songplay_session;"""
drop_query2 = """ DROP TABLE IF EXISTS songplay_user_session;"""
drop_query3 = """ DROP TABLE IF EXISTS user_song_history;"""
session.execute(drop_query1);
session.execute(drop_query2);
session.execute(drop_query3);

### Close the session and cluster connection

In [21]:
session.shutdown()
cluster.shutdown()